In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
sys.path.append('../saildrone/subroutines/')
import warnings
warnings.simplefilter('ignore') # filter some warning messages
from glob import glob


# read in collocated orbital data files

In [ ]:
data_dir = 'F:/data/cruise_data/saildrone/sss/sss_collocations/'
data_dir_out = 'F:/data/cruise_data/saildrone/sss/sss_collocations_norepeat/'
filenames = [x for x in glob(data_dir+'*.nc')]
filenames

In [ ]:
loc=['lat','lon']
for iname,name in enumerate(filenames):
    i = name.find('\\')
    fout = data_dir_out + name[i+1:-3]+'norep.nc' 
    ds = xr.open_dataset(name)
    ds.close()
    
    #if lat/lon in coordinates rather than variables move to variables
    for cor in ds.coords:
        if cor in loc:
            ds = ds.reset_coords(cor)
    
    ds_tem2 = ds.where((ds.smap_SSS<50) & (ds.smap_SSS>1),drop=True)    
    isv=0
    while len(ds_tem2.time)>1:
        i=0
        cond = ((ds_tem2.smap_name==ds_tem2.smap_name[i]) 
                & (ds_tem2.smap_ydim==ds_tem2.smap_ydim[i]) 
                & (ds_tem2.smap_xdim==ds_tem2.smap_xdim[i]))
        subset = ds_tem2.where(cond,drop=True)  #repeat obs
        ds_mn = subset.mean(keep_attrs=True,skipna=True)
        ds_mn['time'] = subset.time.mean()
        ds_mn = ds_mn.assign_coords({'ob':isv})
        if isv==0:
            ds_mn2 = ds_mn
        else:
            ds_mn2 = xr.concat([ds_mn2,ds_mn],dim='ob')
        isv = isv+1
        ds_tem2 = ds_tem2.where(~cond,drop=True)  #data with repeat obs removed
    ds_mn2.to_netcdf(fout)

In [ ]:
#ds= xr.open_dataset('F:/data/sat_data/smap/SSS/L2/JPL/V4.3/2015/136\SMAP_L2B_SSS_01530_20150516T034831_R16010_V4.3.h5')

In [ ]:
file = data_dir_out + 'PMEL_Arctic_2015_sd126-ALL-1_min-v1jplv04.3_filesave4norep.nc'
ds = xr.open_dataset(file)
ds

In [ ]:
file = data_dir_out + 'saildrone-gen_5-arctic_misst_2019-sd1037-20190514T230000-20191011T183000-1_minutes-v1.1575487464625jplv04.3_filesave4norep.nc'
file = data_dir_out + '../sss_collocations/'+'saildrone-gen_5-arctic_misst_2019-sd1037-20190514T230000-20191011T183000-1_minutes-v1.1575487464625jplv04.3_filesave4.nc'
ds = xr.open_dataset(file)
ds

In [ ]:
ds2 = ds.copy(deep=True)
loc=['lat','lon']
for cor in ds.coords:
    if cor in loc:
        print(cor)
        ds2 = ds2.reset_coords(cor)

In [ ]:
ds2

# 8-day averages - remove repeats

In [2]:
data_dir = 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest/'
data_dir_out = 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest_norepeat/'
filenames = [x for x in glob(data_dir+'*.nc')]
filenames

['F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2015_sd126-ALL-1_min-v1_JPL8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2015_sd126-ALL-1_min-v1_RSS8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2015_sd128-ALL-1_min-v1_JPL8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2015_sd128-ALL-1_min-v1_RSS8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2016_sd126-ALL-1_min-v1_JPL8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2016_sd126-ALL-1_min-v1_RSS8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2016_sd128-ALL-1_min-v1_JPL8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\PMEL_Arctic_2016_sd128-ALL-1_min-v1_RSS8dy.nc',
 'F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\\saildrone-gen

In [18]:
print(filenames[1])
ds = xr.open_dataset(filenames[1])
ds.close()
ds

F:/data/cruise_data/saildrone/sss/sss_collocations_8day_nearest\PMEL_Arctic_2015_sd126-ALL-1_min-v1_RSS8dy.nc


<xarray.Dataset>
Dimensions:                       (time: 114373, traj: 1)
Coordinates:
  * traj                          (traj) float64 1.0
    obs                           (time) float64 ...
  * time                          (time) datetime64[ns] 2015-05-12T20:00:16 ....
Data variables:
    lat                           (time) float64 ...
    lon                           (time) float64 ...
    TEMP_AIR_MEAN                 (time) float64 ...
    RH_MEAN                       (time) float64 ...
    BARO_PRES_MEAN                (time) float64 ...
    SAL_CTD_MEAN                  (time) float64 ...
    TEMP_CTD_MEAN                 (time) float64 ...
    TEMP_CTD_STDDEV               (time) float64 ...
    CDOM_MEAN                     (time) float64 ...
    CHLOR_MEAN                    (time) float64 ...
    PAR_AIR_MEAN                  (time) float64 ...
    GUST_WND_MEAN                 (time) float64 ...
    UWND_MEAN                     (time) float64 ...
    VWND_MEAN                     (time) float64 ...
    SOG_MEAN                      (time) float64 ...
    COG_MEAN                      (time) float64 ...
    HDG_MEAN                      (time) float64 ...
    ROLL_MEAN                     (time) float64 ...
    PITCH_MEAN                    (time) float64 ...
    WWND_MEAN                     (time) float64 ...
    TEMP_RBR_MEAN                 (time) float64 ...
    SAL_RBR_MEAN                  (time) float64 ...
    TEMP_O2_RBR_MEAN              (time) float64 ...
    CHLOR_WETLABS_MEAN            (time) float64 ...
    WIND_MEASUREMENT_HEIGHT_MEAN  (time) float64 ...
    sat_nobs                      (time) float64 ...
    sat_nobs_40km                 (time) float64 ...
    sat_sss_smap                  (time) float32 ...
    sat_sss_smap_uncertainty      (time) float32 ...
    sat_sss_smap_40km             (time) float32 ...
    sat_sss_ref                   (time) float32 ...
    sat_gland                     (time) float32 ...
    sat_fland                     (time) float32 ...
    sat_gice                      (time) float32 ...
    sat_surtep                    (time) float32 ...
    sat_lat                       (time) float64 ...
    sat_lon                       (time) float64 ...
Attributes:
    history:      FERRET V7.3  19-Dec-17
    Conventions:  CF-1.6
    featureType:  trajectory
    project:      PMEL_Arctic_2015
    vehicle_id:   sd126

In [ ]:
loc=['lat','lon']
for iname,name in enumerate(filenames):
    print(iname,len(filenames))
    i = name.find('\\')
    fout = data_dir_out + name[i+1:-3]+'norep.nc' 
    ds = xr.open_dataset(name)
    ds.close()
    if 'RSS' in name:  
        ds_tem2 = ds.where((ds.sat_sss_smap<50) & (ds.sat_sss_smap>1),drop=True)    
        isv=0
        while len(ds_tem2.time)>1:
            i=0
            cond = ((ds_tem2.sat_sss_smap==ds_tem2.sat_sss_smap[i]) 
                    & (ds_tem2.sat_sss_smap_uncertainty==ds_tem2.sat_sss_smap_uncertainty[i]) 
                    & (ds_tem2.sat_sss_smap_40km==ds_tem2.sat_sss_smap_40km[i]))
            subset = ds_tem2.where(cond,drop=True)  #repeat obs
            ds_mn = subset.mean(keep_attrs=True,skipna=True)
            ds_mn['time'] = subset.time.mean()
            ds_mn = ds_mn.assign_coords({'ob':isv})
            if isv==0:
                ds_mn2 = ds_mn
            else:
                ds_mn2 = xr.concat([ds_mn2,ds_mn],dim='ob')
            isv = isv+1
            ds_tem2 = ds_tem2.where(~cond,drop=True)  #data with repeat obs removed
    else:
        ds_tem2 = ds.where((ds.sat_smap_sss<50) & (ds.sat_smap_sss>1),drop=True)    
        isv=0
        while len(ds_tem2.time)>1:
            i=0
            cond = ((ds_tem2.sat_smap_sss==ds_tem2.sat_smap_sss[i]) 
                    & (ds_tem2.sat_anc_sst==ds_tem2.sat_anc_sst[i]) 
                    & (ds_tem2.sat_anc_sss==ds_tem2.sat_anc_sss[i]))
            subset = ds_tem2.where(cond,drop=True)  #repeat obs
            ds_mn = subset.mean(keep_attrs=True,skipna=True)
            ds_mn['time'] = subset.time.mean()
            ds_mn = ds_mn.assign_coords({'ob':isv})
            if isv==0:
                ds_mn2 = ds_mn
            else:
                ds_mn2 = xr.concat([ds_mn2,ds_mn],dim='ob')
            isv = isv+1
            ds_tem2 = ds_tem2.where(~cond,drop=True)  #data with repeat obs removed
    ds_mn2.to_netcdf(fout)

0 92
1 92
2 92
3 92
4 92
5 92
6 92
7 92
8 92
9 92
10 92
11 92
12 92
13 92
14 92
15 92
16 92
17 92
18 92
19 92
20 92
21 92
22 92
23 92
24 92
25 92
26 92
27 92
28 92
29 92
30 92
31 92
32 92
33 92
34 92
35 92
36 92
37 92
38 92
39 92
40 92
41 92
42 92
43 92
44 92
45 92
46 92
47 92
48 92
49 92
50 92
51 92
52 92
53 92
54 92


In [ ]:
ds

In [ ]:
ds_mn2

In [ ]:
iname=0
name = filenames[iname]
i = name.find('\\')
fout = data_dir_out + name[i+1:-3]+'norep.nc' 
ds = xr.open_dataset(name)
ds.close()
ds